In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, DoubleType, IntegerType
from pyspark.sql.functions import col, struct, explode, when, lit, array_max, array, split



In [44]:
# vep_srcdir = 's3://dig-analysis-data/out/varianteffect/effects/part-*'
# freq_srcdir = 's3://dig-analysis-data/out/frequencyanalysis/'
# outdir = 's3://dig-bio-index/burden/vepbinning'

# development localhost directories
variant_srcdir = '/Users/mduby/Data/Broad/Magma/Common/part*'
out_dir = '/Users/mduby/Data/Broad/Magma/Out/Step1/'

# print
print("the variant input directory is: {}".format(variant_srcdir))
print("the output directory is: {}".format(out_srcdir))



the variant input directory is: /Users/mduby/Data/Broad/Magma/Common/part*
the output directory is: /Users/mduby/Data/Broad/Magma/Out/Step1/


In [3]:
# open spark session
spark = SparkSession.builder.appName('bioindex').getOrCreate()

print("got Spark session of type {}".format(type(spark)))


got Spark session of type <class 'pyspark.sql.session.SparkSession'>


In [5]:
# this is the schema for the common variant file
variant_schema = StructType(
    [
        StructField('varId', StringType(), nullable=False),
        StructField('dbSNP', StringType(), nullable=False),
        StructField('consequence', StringType(), nullable=False),
        StructField('gene', StringType(), nullable=False),
        StructField('transcript', StringType(), nullable=False),
        StructField('impact', StringType(), nullable=False),
    ]
)


In [15]:
# method to load the frequencies
df_load = spark.read \
        .csv(variant_srcdir, sep='\t', header=True, schema=variant_schema) \
        .select('varId', 'dbSNP')# method to load the frequencies

# print
print("the loaded variant data frame has {} rows".format(df_load.count()))
df_load.show()
        

the loaded variant data frame has 1465865 rows
+---------------+------------+
|          varId|       dbSNP|
+---------------+------------+
|1:66159416:T:TC|        null|
| 1:66160075:T:A|   rs6664534|
| 1:66167207:G:A| rs544422243|
| 1:66181653:C:T|rs1045050806|
| 1:66184360:T:C| rs569568659|
| 1:66189332:A:G| rs186428007|
| 1:66203588:A:T| rs374304719|
| 1:66210208:C:T| rs560512594|
|  1:6621737:C:T| rs772700121|
| 1:66247464:G:A| rs192224545|
| 1:66257069:A:G| rs988459270|
| 1:66281675:G:T|  rs72931395|
| 1:66284502:G:A| rs567848519|
| 1:66289594:A:T|  rs12021998|
| 1:66291395:A:G| rs566719052|
|  1:6631086:T:A|        null|
| 1:66312242:A:G| rs113234023|
| 1:66324905:T:A| rs543414819|
| 1:66330519:C:T| rs565829829|
|  1:6633230:G:A|  rs12132145|
+---------------+------------+
only showing top 20 rows



In [17]:
# keep only the rows with non null dbSNP ids
df_nonnull_load = df_load.filter(col("dbSNP").isNotNull())

# print
print("the non null RS id dataframe has {} rows".format(df_nonnull_load.count()))

the non null RS id dataframe has 1295699 rows


In [18]:
df_nonnull_load.show()

+--------------+------------+
|         varId|       dbSNP|
+--------------+------------+
|1:66160075:T:A|   rs6664534|
|1:66167207:G:A| rs544422243|
|1:66181653:C:T|rs1045050806|
|1:66184360:T:C| rs569568659|
|1:66189332:A:G| rs186428007|
|1:66203588:A:T| rs374304719|
|1:66210208:C:T| rs560512594|
| 1:6621737:C:T| rs772700121|
|1:66247464:G:A| rs192224545|
|1:66257069:A:G| rs988459270|
|1:66281675:G:T|  rs72931395|
|1:66284502:G:A| rs567848519|
|1:66289594:A:T|  rs12021998|
|1:66291395:A:G| rs566719052|
|1:66312242:A:G| rs113234023|
|1:66324905:T:A| rs543414819|
|1:66330519:C:T| rs565829829|
| 1:6633230:G:A|  rs12132145|
|1:66333368:A:G| rs753963099|
|1:66348745:T:G| rs927568542|
+--------------+------------+
only showing top 20 rows



In [21]:
# decompose first field and get chrom/pos
split_col = split(df_nonnull_load['varId'], ':')

# add the first two columns back in
df_nonnull_load = df_nonnull_load.withColumn('chromosome', split_col.getItem(0))
df_nonnull_load = df_nonnull_load.withColumn('position', split_col.getItem(1))



AttributeError: 'DataFrame' object has no attribute 'shaow'

In [22]:
df_nonnull_load.show()

+--------------+------------+----------+--------+
|         varId|       dbSNP|chromosome|position|
+--------------+------------+----------+--------+
|1:66160075:T:A|   rs6664534|         1|66160075|
|1:66167207:G:A| rs544422243|         1|66167207|
|1:66181653:C:T|rs1045050806|         1|66181653|
|1:66184360:T:C| rs569568659|         1|66184360|
|1:66189332:A:G| rs186428007|         1|66189332|
|1:66203588:A:T| rs374304719|         1|66203588|
|1:66210208:C:T| rs560512594|         1|66210208|
| 1:6621737:C:T| rs772700121|         1| 6621737|
|1:66247464:G:A| rs192224545|         1|66247464|
|1:66257069:A:G| rs988459270|         1|66257069|
|1:66281675:G:T|  rs72931395|         1|66281675|
|1:66284502:G:A| rs567848519|         1|66284502|
|1:66289594:A:T|  rs12021998|         1|66289594|
|1:66291395:A:G| rs566719052|         1|66291395|
|1:66312242:A:G| rs113234023|         1|66312242|
|1:66324905:T:A| rs543414819|         1|66324905|
|1:66330519:C:T| rs565829829|         1|66330519|


In [25]:
# build out data frame and save magma variant input file
df_export = df_nonnull_load.select("dbSnp", 'chromosome', 'position')



In [26]:
df_export.show()

+------------+----------+--------+
|       dbSnp|chromosome|position|
+------------+----------+--------+
|   rs6664534|         1|66160075|
| rs544422243|         1|66167207|
|rs1045050806|         1|66181653|
| rs569568659|         1|66184360|
| rs186428007|         1|66189332|
| rs374304719|         1|66203588|
| rs560512594|         1|66210208|
| rs772700121|         1| 6621737|
| rs192224545|         1|66247464|
| rs988459270|         1|66257069|
|  rs72931395|         1|66281675|
| rs567848519|         1|66284502|
|  rs12021998|         1|66289594|
| rs566719052|         1|66291395|
| rs113234023|         1|66312242|
| rs543414819|         1|66324905|
| rs565829829|         1|66330519|
|  rs12132145|         1| 6633230|
| rs753963099|         1|66333368|
| rs927568542|         1|66348745|
+------------+----------+--------+
only showing top 20 rows



In [46]:
# write out the tab delimited file
df_export.write.mode('overwrite').option("delimiter", "\t").csv(out_dir)

In [ ]:
# coalesce into a single file
